In [149]:
import bigframes.pandas as bpd
import bigframes.ml.llm as llm
import numpy as np

In [150]:
import numpy as np
import pandas as pd

In [151]:
pip install "datasets<3.6.0"

In [152]:
from datasets import load_dataset

In [153]:
ds_2020 = load_dataset("c3po-ai/edgar-corpus", "year_2020", split="test")

In [154]:
ds_2020

Dataset({
    features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
    num_rows: 685
})

In [155]:
import psutil

# Check overall system memory
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / 1e9:.2f} GB")
print(f"Available RAM: {ram.available / 1e9:.2f} GB")
print(f"Used RAM: {ram.used / 1e9:.2f} GB")
print(f"RAM usage: {ram.percent}%")

Total RAM: 16.78 GB
Available RAM: 12.75 GB
Used RAM: 3.64 GB
RAM usage: 24.0%


In [156]:
ds_streaming = load_dataset("c3po-ai/edgar-corpus", "full", streaming=True)

In [157]:
ds_2020

Dataset({
    features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
    num_rows: 685
})

In [158]:
ds_streaming

IterableDatasetDict({
    train: IterableDataset({
        features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
        num_shards: 28
    })
    validation: IterableDataset({
        features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
        num_shards: 28
    })
    test: IterableDataset({
        features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8',

In [159]:
# ds_streaming_train = pd.DataFrame(ds_streaming["train"])

In [160]:
df_cik_train = pd.DataFrame(columns=['cik', 'year'])

In [161]:
# for row in ds_streaming["train"]:
#     df_cik_train.loc[len(df_cik_train)] = [row['cik'], row['year']]

In [162]:
%%bigquery cik_with_ids
select entities.cik, id from `financial-fraud-detection-1.sec_enforcement_actions.sec_actions`, UNNEST(entities) as entities where entities.cik is not null;

Query is running:   0%|          |

Downloading:   0%|          |

In [163]:
%%bigquery unique_cik_with_counts
SELECT
  entities.cik,
  COUNT(*) AS occurrences
FROM
  `financial-fraud-detection-1.sec_enforcement_actions.sec_actions`,
  UNNEST(entities) AS entities
WHERE
  entities.cik IS NOT NULL
GROUP BY
  entities.cik
ORDER BY
  occurrences DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [164]:
%%bigquery unique_cik
SELECT DISTINCT
  entities.cik
FROM
  `financial-fraud-detection-1.sec_enforcement_actions.sec_actions`,
  UNNEST(entities) AS entities
WHERE
  entities.cik IS NOT NULL;

Query is running:   0%|          |

Downloading:   0%|          |

In [165]:
cik_with_ids

,cik,id
0,888295,f4aea4e72038d9861c4c674839d8a81a
1,64040,ef797b83d0c4d82bbbab26bd4d4e9b94
2,1841857,21e1a637280eeaa1d4635eaa6d4cb31b
3,1332784,61ebd73c0e86a04232b6072c8fe47037
4,23197,4b317f4b821710fb1625495efc318a34
...,...,...
1640,1105705,df29408bfa136f82bf22d4a6a46e30ee
1641,1037949,205905c4d9a85a9c79c2168341855286
1642,1320168,2b3e276ed2e6074aaf5077156ef520fa
1643,765258,b1f24d87703ff49be864662ee2249720


In [166]:
unique_cik_with_counts

,cik,occurrences
0,895421,13
1,1580308,11
2,1572694,11
3,822581,10
4,65100,10
...,...,...
1191,1140334,1
1192,1105705,1
1193,1037949,1
1194,1320168,1


In [167]:
unique_cik

,cik
0,888295
1,64040
2,1841857
3,1332784
4,23197
...,...
1191,1140334
1192,1105705
1193,1037949
1194,1320168


In [168]:
type(unique_cik)

pandas.core.frame.DataFrame

In [169]:
cik_with_ids.duplicated().sum()

np.int64(11)

In [170]:
cik_with_ids[cik_with_ids.duplicated()]

,cik,id
318,1389874,2cedb0052758d5e9e2f1ab021ffd57e3
696,1419186,bac8ad14f24d49e1308d372e90a5dace
710,1381197,384fbe7e12ddb4e79c13c0c7783eddb1
835,1419186,69c3539edafa2d8d78e280bad21227f7
938,1481792,395fc7349c72b97f74f6c80b257cda6a
942,1462284,652d40389eefb2f1dbf5a7befc295c16
1025,1011006,bb7baeb5de7f7c367df40c5757a8a700
1188,1748930,575301b71b3491551095cd087da55933
1425,845982,4e68db4dcdee5b2d04e8da605a50f0ea
1452,1084876,b58dde917ce03b47d9b12b2689dc1471


In [171]:
target_ciks_set = set(unique_cik["cik"].astype(str).tolist())

In [172]:
target_ciks_set

{'1762470',
 '1679788',
 '722830',
 '1707790',
 '77155',
 '1846644',
 '919869',
 '1023291',
 '1965702',
 '1289419',
 '721371',
 '1932998',
 '1137091',
 '1611110',
 '930184',
 '1546383',
 '1327567',
 '931328',
 '28372',
 '724683',
 '1640680',
 '1510295',
 '1602237',
 '1355042',
 '921030',
 '1024452',
 '49728',
 '922475',
 '1012140',
 '1384305',
 '909724',
 '825293',
 '1783317',
 '1333978',
 '1352278',
 '1294250',
 '919533',
 '916085',
 '80424',
 '1874315',
 '1167711',
 '1376339',
 '813762',
 '937394',
 '808450',
 '101984',
 '773468',
 '1901146',
 '1963860',
 '1605301',
 '1517496',
 '1666746',
 '63276',
 '732717',
 '72971',
 '1103390',
 '908255',
 '1095981',
 '757657',
 '1334388',
 '1332784',
 '1037949',
 '884888',
 '1842357',
 '1927285',
 '102754',
 '64040',
 '841692',
 '1237746',
 '1258655',
 '897070',
 '1091748',
 '351346',
 '107136',
 '818686',
 '1821289',
 '1349436',
 '1140486',
 '1057083',
 '1432539',
 '1449732',
 '932540',
 '1280058',
 '1973130',
 '1420816',
 '2019663',
 '1121914'

In [173]:
filtered_ds_train = ds_streaming["train"].filter(lambda row: str(row.get("cik")) in target_ciks_set)

In [174]:
df_2020 = pd.DataFrame(ds_2020)

In [175]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    685 non-null    object
 1   cik         685 non-null    object
 2   year        685 non-null    object
 3   section_1   685 non-null    object
 4   section_1A  685 non-null    object
 5   section_1B  685 non-null    object
 6   section_2   685 non-null    object
 7   section_3   685 non-null    object
 8   section_4   685 non-null    object
 9   section_5   685 non-null    object
 10  section_6   685 non-null    object
 11  section_7   685 non-null    object
 12  section_7A  685 non-null    object
 13  section_8   685 non-null    object
 14  section_9   685 non-null    object
 15  section_9A  685 non-null    object
 16  section_9B  685 non-null    object
 17  section_10  685 non-null    object
 18  section_11  685 non-null    object
 19  section_12  685 non-null    object
 20  section_13

In [176]:
filtered_ds_train

IterableDataset({
    features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
    num_shards: 28
})

In [177]:
# df_filtered_train = pd.DataFrame(filtered_ds_train)

In [178]:
from itertools import islice

In [179]:
islice(filtered_ds_train, 10)

In [180]:
df_filtered_train_10 = pd.DataFrame(islice(filtered_ds_train, 10))

In [181]:
df_filtered_train_10

,filename,cik,year,section_1,section_1A,section_1B,section_2,section_3,section_4,section_5,...,section_8,section_9,section_9A,section_9B,section_10,section_11,section_12,section_13,section_14,section_15
0,10456_1993.txt,10456,1993,ITEM 1. BUSINESS.\n(a) GENERAL DEVELOPMENT OF ...,,,ITEM 2. PROPERTIES.\nThe Company owns or has l...,"ITEM 3. LEGAL PROCEEDINGS.\nAs of December 31,...",ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,,,,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
1,315189_1993.txt,315189,1993,ITEM 1. BUSINESS.\nPRODUCTS\nDeere & Company (...,,,"ITEM 2. PROPERTIES.\nSee ""Manufacturing"" in It...",ITEM 3. LEGAL PROCEEDINGS.\nThe Company is sub...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION.\nThe informat...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
2,36270_1993.txt,36270,1993,Item 1. Business. -------- First Empire State ...,,,Item 2. Properties. ----------\nBoth First Emp...,Item 3. Legal Proceedings. -----------------\n...,Item 4. Submission of Matters to a Vote of Sec...,Item 5. Market for Registrant's Common Equity ...,...,Item 8. Financial Statements and Supplementary...,Item 9. Changes In and Disagreements With Acco...,,,Item 10. Directors and Executive Officers of t...,Item 11. Executive Compensation. Incorporated ...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,"Item 14. Exhibits, Financial Statement Schedul...",
3,73124_1993.txt,73124,1993,Item 1 - -Business\nNORTHERN TRUST CORPORATION...,,,ITEM 2 - -PROPERTIES\nThe executive offices of...,ITEM 3 - -LEGAL PROCEEDINGS\nThe information c...,ITEM 4 - -SUBMISSION OF MATTERS TO A VOTE OF S...,ITEM 5 - -MARKET FOR REGISTRANT'S COMMON EQUIT...,...,ITEM 8 - -FINANCIAL STATEMENTS AND SUPPLEMENTA...,ITEM 9 - -CHANGES IN AND DISAGREEMENTS WITH AC...,,,ITEM 10 - -DIRECTORS AND EXECUTIVE OFFICERS OF...,ITEM 11 - -EXECUTIVE COMPENSATION\nThe informa...,ITEM 12 - -SECURITY OWNERSHIP OF CERTAIN BENEF...,ITEM 13 - -CERTAIN RELATIONSHIPS AND RELATED T...,"Item 14 - -Exhibits, Financial Statement Sched...",
4,52466_1993.txt,52466,1993,"Item 1. BUSINESS\nIonics, Incorporated (""Ionic...",,,Item 2. PROPERTIES\nThe Company owns or leases...,Item 3. LEGAL PROCEEDINGS\nThe Company is from...,Item 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,Item 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,Item 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,Item 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,Item 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,Item 11. EXECUTIVE COMPENSATION\nThe informati...,Item 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,Item 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"Item 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
5,47129_1993.txt,47129,1993,ITEM 1. BUSINESS.\nGeneral\nThe Hertz Corporat...,,,ITEM 2. PROPERTIES.\nHertz' operations are car...,ITEM 3. LEGAL PROCEEDINGS.\nVarious legal acti...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION.\nThe followin...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
6,72971_1993.txt,72971,1993,ITEM 1. BUSINESS Norwest Corporation (the corp...,,,ITEM 2. PROPERTIES 

In [182]:
unique_cik.loc[unique_cik['cik'] == 10456]

,cik
688,10456


In [183]:
# from collections import deque

# def last_n(iterable, n):
#     return deque(iterable, maxlen=n)

# # Usage
# df_filtered_train_10 = last_n(filtered_ds_train, 10)  # buffers last 10 rows
# for row in df_filtered_train_10:
#     print(row)

In [184]:
slice_10_20 = islice(filtered_ds_train, 10, 20)

In [185]:
df_filtered_train_next_10 = pd.DataFrame(slice_10_20)

In [186]:
df_filtered_train_next_10

,filename,cik,year,section_1,section_1A,section_1B,section_2,section_3,section_4,section_5,...,section_8,section_9,section_9A,section_9B,section_10,section_11,section_12,section_13,section_14,section_15
0,1800_1993.txt,1800,1993,ITEM 1. BUSINESS\nGENERAL DEVELOPMENT OF BUSIN...,,,ITEM 2. PROPERTIES\nThe Company's corporate of...,ITEM 3. LEGAL PROCEEDINGS\nThe Company is invo...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. DISAGREEMENTS ON ACCOUNTING AND FINANC...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION\nThe material ...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
1,310158_1993.txt,310158,1993,"Item 1. Business\nGeneral\nSchering-Plough*, i...",,,Item 2. Properties\nThe Company's corporate he...,Item 3. Legal Proceedings\nSchering Corporatio...,Item 4. Submission of Matters to a Vote of Sec...,Item 5.Market for Registrant's Common Equity a...,...,Item 8.Financial Statements and Supplementary ...,Item 9.Changes in and Disagreements with Accou...,,,Item 10.Directors and Executive Officers of th...,Item 11.Executive Compensation\nExecutive comp...,Item 12.Security Ownership of Certain Benefici...,Item 13.Certain Relationships and Related Tran...,"Item 14.Exhibits, Financial Statement Schedule...",
2,51143_1993.txt,51143,1993,"ITEM 1. BUSINESS: IBM develops, manufactures a...",,,"ITEM 2. PROPERTIES: At December 31, 1993, IBM'...",ITEM 3. LEGAL PROCEEDINGS: No material pending...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION:\nRefer to pag...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
3,63276_1993.txt,63276,1993,Item 1. Business - ------- --------\nThe Compa...,,,Item 2. Properties - ------- ----------\nThe C...,Item 3. Legal Proceedings - ------- ----------...,Item 4. Submission of Matters to a Vote of Sec...,Item 5. Market for the Registrant's Common Equ...,...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,,,Item 10. Directors and Executive Officers of t...,Item 11. Executive Compensation - -------- ---...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,"Item 14. Exhibits, Financial Statements, and R...",
4,51644_1993.txt,51644,1993,Item 1. Business\nThe Interpublic Group of Com...,,,Item 2. Properties\nMost of the advertising op...,Item 3. Legal Proceedings\nTwo of the Company'...,Item 4. Submission of Matters to a Vote of Sec...,Item 5. Market for the Registrant's Common Equ...,...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,,,Item 10. Directors and Executive Officers of t...,Item 11. Executive Compensation\nThe informati...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,"Item 14. Exhibits, Financial Statement Schedul...",
5,75208_1993.txt,75208,1993,"ITEM 1. BUSINESS\nOverseas Shipholding Group, ...",,,ITEM 2. PROPERTIES\n---------- See Item 1.\nIT...,ITEM 3. LEGAL PROCEEDINGS ----------------- Th...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5.Market for Registrant's Last three para...,...,"ITEM 8.Financial Statements and ""Consolidated ...",ITEM 9.Changes in and Disagreements with Accou...,,,"ITEM 10. Directors and Executive ""Election of ...","ITEM 11. Executive Compensation ""Compensation ...","ITEM 12. Security Ownership of ""Election of Di...","ITEM 13. Certain Relationships and ""Election

In [187]:
unique_cik.loc[unique_cik['cik'] == 1800]

,cik
193,1800


In [188]:
filtered_ds_train

IterableDataset({
    features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
    num_shards: 28
})

In [189]:
# filtered_ds_train_2020 = filtered_ds_train.filter(lambda row: row.get("year") == "2020")

In [190]:
shard_27 = filtered_ds_train.shard(num_shards=filtered_ds_train.num_shards, index=27)

In [191]:
shard_27

IterableDataset({
    features: ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B', 'section_2', 'section_3', 'section_4', 'section_5', 'section_6', 'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A', 'section_9B', 'section_10', 'section_11', 'section_12', 'section_13', 'section_14', 'section_15'],
    num_shards: 1
})

In [192]:
shard_27_10 = islice(shard_27, 10)

In [193]:
df_last_shard = pd.DataFrame(shard_27_10)

In [194]:
df_last_shard

,filename,cik,year,section_1,section_1A,section_1B,section_2,section_3,section_4,section_5,...,section_8,section_9,section_9A,section_9B,section_10,section_11,section_12,section_13,section_14,section_15
0,7084_2020.htm,7084,2020,Item 1.BUSINESS (Continued)\nAg Services and O...,Item 1A.RISK FACTORS (Continued)\nGlobal Opera...,Item 1B.UNRESOLVED STAFF COMMENTS\nThe Company...,Item 2.PROPERTIES (Continued)\n*The U.S. proce...,Item 3.LEGAL PROCEEDINGS\nThe Company is routi...,Item 4.MINE SAFETY DISCLOSURES\nNone.\nPART II...,"Item 5.MARKET FOR REGISTRANT’S COMMON EQUITY, ...",...,Item 8.FINANCIAL STATEMENTS AND SUPPLEMENTARY ...,Item 9.CHANGES IN AND DISAGREEMENTS WITH ACCOU...,Item 9A.CONTROLS AND PROCEDURES\nAs of Decembe...,Item 9B.OTHER INFORMATION\nNone.\nPART III\nIt...,"Item 10.DIRECTORS, EXECUTIVE OFFICERS AND CORP...",,Item 12.SECURITY OWNERSHIP OF CERTAIN BENEFICI...,Item 13.CERTAIN RELATIONSHIPS AND RELATED TRAN...,Item 14.PRINCIPAL ACCOUNTING FEES AND SERVICES...,Item 15.EXHIBITS AND FINANCIAL STATEMENT SCHED...
1,1067983_2020.htm,1067983,2020,Item 1. Business Description\nBerkshire Hathaw...,Item 1A. Risk Factors\nBerkshire and its subsi...,Item 1B. Unresolved Staff Comments\nNone.\nIte...,Item 2. Description of Properties\nThe propert...,Item 3. Legal Proceedings\nBerkshire and its s...,Item 4. Mine Safety Disclosures\nInformation r...,"Item 5. Market for Registrant’s Common Equity,...",...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,Item 9A. Controls and Procedures\nAt the end o...,Item 9B. Other Information\nNone\nPart III\nEx...,,,,,,Item 15. Exhibits and Financial Statement Sche...
2,8868_2020.htm,8868,2020,"ITEM 1. BUSINESS\n(U.S. dollars in millions, e...",ITEM 1A. RISK FACTORS\nYou should carefully co...,ITEM 1B. UNRESOLVED STAFF COMMENTS\nNot applic...,ITEM 2. PROPERTIES\nOur principal properties w...,ITEM 3. LEGAL PROCEEDINGS\nReference is made t...,ITEM 4. MINE SAFETY DISCLOSURES\nNot applicabl...,"ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY,...",...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,ITEM 9A. CONTROLS AND PROCEDURES\nEvaluation o...,ITEM 9B. OTHER INFORMATION\nNot applicable.\nP...,"ITEM 10. DIRECTORS, EXECUTIVE OFFICERS AND COR...",ITEM 11. EXECUTIVE COMPENSATION\nThe informati...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,ITEM 14. PRINCIPAL ACCOUNTANT FEES AND SERVICE...,ITEM 15.EXHIBITS AND FINANCIAL STATEMENT SCHED...
3,1137091_2020.htm,1137091,2020,Item 1. Business.\nGeneral Business Overview\n...,Item 1A. Risk Factors.\nThe Company’s business...,Item 1B. Unresolved Staff Comments.\nNone.\nIt...,Item 2. Properties.\nThe Company’s operations ...,Item 3. Legal Proceedings.\nSee Note 10. Commi...,Item 4. Mine Safety Disclosures.\nNot applicab...,"Item 5. Market for Registrant’s Common Equity,...",...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,Item 9A. Controls and Procedures.\nEvaluation ...,Item 9B. Other Information.\nNone.\nPART III\n...,"Item 10. Directors, Executive Officers and Cor...",Item 11. Executive Compensation.\nThe informat...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,Item 14. Principal Accounting Fees and Service...,"Item 15. Exhibits, Financial Statement Schedul..."
4,316709_2020.htm,316709,2020,Item 1. Business\nGeneral Corporate Overview\n...,Item 1A. Risk Factors\nWe face a variety of ri...,Item 1B. Unresolved Staff Comments\nNone.\n- 2...,Item 2. Properties\nAs part of our real estate...,Item 3. Legal Proceedings\nFor a discussion of...,Item 4. Mine Safety Disclosures\nNot applicabl...,"Item 5. Market for Registrant’s Common Equity,...",...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements With Acco...,Item 9A. Controls and Procedures\nEvaluation o...,Item 

In [195]:
df_edgar = pd.concat([df_filtered_train_10, df_filtered_train_next_10, df_last_shard], ignore_index=True)

In [196]:
df_edgar

,filename,cik,year,section_1,section_1A,section_1B,section_2,section_3,section_4,section_5,...,section_8,section_9,section_9A,section_9B,section_10,section_11,section_12,section_13,section_14,section_15
0,10456_1993.txt,10456,1993,ITEM 1. BUSINESS.\n(a) GENERAL DEVELOPMENT OF ...,,,ITEM 2. PROPERTIES.\nThe Company owns or has l...,"ITEM 3. LEGAL PROCEEDINGS.\nAs of December 31,...",ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,,,,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
1,315189_1993.txt,315189,1993,ITEM 1. BUSINESS.\nPRODUCTS\nDeere & Company (...,,,"ITEM 2. PROPERTIES.\nSee ""Manufacturing"" in It...",ITEM 3. LEGAL PROCEEDINGS.\nThe Company is sub...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION.\nThe informat...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
2,36270_1993.txt,36270,1993,Item 1. Business. -------- First Empire State ...,,,Item 2. Properties. ----------\nBoth First Emp...,Item 3. Legal Proceedings. -----------------\n...,Item 4. Submission of Matters to a Vote of Sec...,Item 5. Market for Registrant's Common Equity ...,...,Item 8. Financial Statements and Supplementary...,Item 9. Changes In and Disagreements With Acco...,,,Item 10. Directors and Executive Officers of t...,Item 11. Executive Compensation. Incorporated ...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,"Item 14. Exhibits, Financial Statement Schedul...",
3,73124_1993.txt,73124,1993,Item 1 - -Business\nNORTHERN TRUST CORPORATION...,,,ITEM 2 - -PROPERTIES\nThe executive offices of...,ITEM 3 - -LEGAL PROCEEDINGS\nThe information c...,ITEM 4 - -SUBMISSION OF MATTERS TO A VOTE OF S...,ITEM 5 - -MARKET FOR REGISTRANT'S COMMON EQUIT...,...,ITEM 8 - -FINANCIAL STATEMENTS AND SUPPLEMENTA...,ITEM 9 - -CHANGES IN AND DISAGREEMENTS WITH AC...,,,ITEM 10 - -DIRECTORS AND EXECUTIVE OFFICERS OF...,ITEM 11 - -EXECUTIVE COMPENSATION\nThe informa...,ITEM 12 - -SECURITY OWNERSHIP OF CERTAIN BENEF...,ITEM 13 - -CERTAIN RELATIONSHIPS AND RELATED T...,"Item 14 - -Exhibits, Financial Statement Sched...",
4,52466_1993.txt,52466,1993,"Item 1. BUSINESS\nIonics, Incorporated (""Ionic...",,,Item 2. PROPERTIES\nThe Company owns or leases...,Item 3. LEGAL PROCEEDINGS\nThe Company is from...,Item 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,Item 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,Item 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,Item 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,Item 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,Item 11. EXECUTIVE COMPENSATION\nThe informati...,Item 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,Item 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"Item 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
5,47129_1993.txt,47129,1993,ITEM 1. BUSINESS.\nGeneral\nThe Hertz Corporat...,,,ITEM 2. PROPERTIES.\nHertz' operations are car...,ITEM 3. LEGAL PROCEEDINGS.\nVarious legal acti...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION.\nThe followin...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
6,72971_1993.txt,72971,1993,ITEM 1. BUSINESS Norwest Corporation (the corp...,,,ITEM 2. PROPERTIES 

In [197]:
# check if any cik starts with 0 in trainset, if not we can make cik as integer

In [198]:
filtered_ds_train_starts0 = ds_streaming["train"].filter(lambda row: str(row.get("cik")).startswith("0"))

In [199]:
slice_10_start0 = islice(filtered_ds_train_starts0, 10)

In [201]:
df_start_with_0_10 = pd.DataFrame(slice_10_start0)

In [202]:
df_start_with_0_10

""


In [203]:
# cik not starting with 0 so we can convert it to int

In [208]:
df_edgar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    30 non-null     object
 1   cik         30 non-null     int64 
 2   year        30 non-null     int64 
 3   section_1   30 non-null     object
 4   section_1A  30 non-null     object
 5   section_1B  30 non-null     object
 6   section_2   30 non-null     object
 7   section_3   30 non-null     object
 8   section_4   30 non-null     object
 9   section_5   30 non-null     object
 10  section_6   30 non-null     object
 11  section_7   30 non-null     object
 12  section_7A  30 non-null     object
 13  section_8   30 non-null     object
 14  section_9   30 non-null     object
 15  section_9A  30 non-null     object
 16  section_9B  30 non-null     object
 17  section_10  30 non-null     object
 18  section_11  30 non-null     object
 19  section_12  30 non-null     object
 20  section_13  

In [209]:
df_edgar['cik'] = df_edgar['cik'].astype(int)

In [210]:
df_edgar['year'] = df_edgar['year'].astype(int)

In [211]:
df_edgar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    30 non-null     object
 1   cik         30 non-null     int64 
 2   year        30 non-null     int64 
 3   section_1   30 non-null     object
 4   section_1A  30 non-null     object
 5   section_1B  30 non-null     object
 6   section_2   30 non-null     object
 7   section_3   30 non-null     object
 8   section_4   30 non-null     object
 9   section_5   30 non-null     object
 10  section_6   30 non-null     object
 11  section_7   30 non-null     object
 12  section_7A  30 non-null     object
 13  section_8   30 non-null     object
 14  section_9   30 non-null     object
 15  section_9A  30 non-null     object
 16  section_9B  30 non-null     object
 17  section_10  30 non-null     object
 18  section_11  30 non-null     object
 19  section_12  30 non-null     object
 20  section_13  

In [212]:
from google.cloud import bigquery

In [213]:
client = bigquery.Client()

In [214]:
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_EMPTY",
)

table_id = "financial-fraud-detection-1.EDGAR.edgar_corpus_sample"

job = client.load_table_from_dataframe(
    df_edgar, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.



Conflict: 409 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/financial-fraud-detection-1/jobs?uploadType=multipart: Already Exists: Table financial-fraud-detection-1:EDGAR.edgar_corpus_sample

In [215]:
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 30 rows and 23 columns to financial-fraud-detection-1.EDGAR.edgar_corpus_sample


In [216]:
df_edgar_small = pd.concat([df_filtered_train_10.sample(n=1), df_filtered_train_next_10.sample(n=2), df_last_shard.sample(n=2)], ignore_index=True)

In [217]:
df_edgar_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    5 non-null      object
 1   cik         5 non-null      object
 2   year        5 non-null      object
 3   section_1   5 non-null      object
 4   section_1A  5 non-null      object
 5   section_1B  5 non-null      object
 6   section_2   5 non-null      object
 7   section_3   5 non-null      object
 8   section_4   5 non-null      object
 9   section_5   5 non-null      object
 10  section_6   5 non-null      object
 11  section_7   5 non-null      object
 12  section_7A  5 non-null      object
 13  section_8   5 non-null      object
 14  section_9   5 non-null      object
 15  section_9A  5 non-null      object
 16  section_9B  5 non-null      object
 17  section_10  5 non-null      object
 18  section_11  5 non-null      object
 19  section_12  5 non-null      object
 20  section_13  5 

In [218]:
df_edgar_small['cik'] = df_edgar_small['cik'].astype(int)

In [219]:
df_edgar_small['year'] = df_edgar_small['year'].astype(int)

In [220]:
df_edgar_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    5 non-null      object
 1   cik         5 non-null      int64 
 2   year        5 non-null      int64 
 3   section_1   5 non-null      object
 4   section_1A  5 non-null      object
 5   section_1B  5 non-null      object
 6   section_2   5 non-null      object
 7   section_3   5 non-null      object
 8   section_4   5 non-null      object
 9   section_5   5 non-null      object
 10  section_6   5 non-null      object
 11  section_7   5 non-null      object
 12  section_7A  5 non-null      object
 13  section_8   5 non-null      object
 14  section_9   5 non-null      object
 15  section_9A  5 non-null      object
 16  section_9B  5 non-null      object
 17  section_10  5 non-null      object
 18  section_11  5 non-null      object
 19  section_12  5 non-null      object
 20  section_13  5 

In [221]:
table_id_small = "financial-fraud-detection-1.EDGAR.edgar_corpus_sample_small"

job = client.load_table_from_dataframe(
    df_edgar_small, table_id_small, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.


LoadJob<project=financial-fraud-detection-1, location=US, id=bc665bb3-ac30-4a85-9f95-3c563a414ab0>

In [223]:
table_small = client.get_table(table_id_small)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table_small.num_rows, len(table_small.schema), table_id_small
    )
)

Loaded 5 rows and 23 columns to financial-fraud-detection-1.EDGAR.edgar_corpus_sample_small
